In [ ]:
import os
import firebase_admin
from firebase_admin import credentials, firestore
from dotenv import load_dotenv

In [20]:
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Access variables
firebase_key_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
project_id = os.getenv("FIRESTORE_PROJECT_ID")
database_url = os.getenv("FIRESTORE_DATABASE_URL")

print("Firebase Key Path:", firebase_key_path)
print("Firestore Project ID:", project_id)
print("Firestore Database URL:", database_url)


Firebase Key Path: .streamlit/tournalytics-firebase-adminsdk-dbcy8-e1435aaaa6.json
Firestore Project ID: tournalytics
Firestore Database URL: https://firestore.googleapis.com/v1/projects/tournalytics/databases/(default)/documents


In [21]:
import firebase_admin
from firebase_admin import credentials, firestore

# Initialize Firebase Admin SDK
try:
    if not firebase_admin._apps:
        cred = credentials.Certificate(firebase_key_path)
        firebase_admin.initialize_app(cred, {"projectId": project_id})
    print("Firebase initialized successfully!")
except Exception as e:
    print("Error initializing Firebase:", e)


Firebase initialized successfully!


In [22]:
# Test Firestore connection
try:
    db = firestore.client()
    users_ref = db.collection("users")
    docs = users_ref.stream()
    
    print("Users collection documents:")
    for doc in docs:
        print(doc.id, doc.to_dict())
except Exception as e:
    print("Error connecting to Firestore:", e)


Users collection documents:
dmf95 {'created_at': DatetimeWithNanoseconds(2025, 1, 6, 15, 48, 46, 161429, tzinfo=datetime.timezone.utc), 'email': 'domenicfayad@gmail.com', 'league_id': None, 'username': 'dmf95', 'role': 'super_admin'}


In [26]:
from firebase_admin import firestore
from datetime import datetime

# Initialize Firestore client
db = firestore.client()

# Define metadata for the user
email = "domenicfayad@gmail.com"
username = "dmf95"  # Extracted or defined username
user_metadata = {
    "username": username,
    "email": email,
    "role": "super_admin",  # Replace with appropriate role ('user', 'admin', 'super_admin')
    "league_id": None,  # Add associated league if applicable
    "created_at": datetime.now(),
}

# Save the metadata in the Firestore `users` collection
try:
    db.collection("users").document(username).set(user_metadata)
    print(f"User metadata for '{username}' saved in Firestore.")
except Exception as e:
    print("Error saving user metadata:", e)


User metadata for 'dmf95' saved in Firestore.


In [ ]:
def create_user_metadata(username, password, email, role="user", league_id=None):
    """
    Create a new user in Firestore with metadata.
    """
    try:
        # Hash the password
        hashed_password = bcrypt.hashpw(password.encode("utf-8"), bcrypt.gensalt()).decode("utf-8")

        # Create user metadata
        user_doc = {
            "username": username,
            "hashed_password": hashed_password,
            "email": email,
            "role": role,
            "league_id": league_id,
            "created_at": firestore.SERVER_TIMESTAMP,
        }

        # Add user to Firestore
        db.collection("users").document(username).set(user_doc)

        # Create user in Firebase Auth for email authentication
        auth.create_user(email=email, display_name=username)

        print(f"User {username} created successfully.")
        return True
    except Exception as e:
        print(f"Error creating user metadata: {e}")
        return False

# Test creating a new user
create_user_metadata(
    username="dmf95",
    password="password123",
    email="testuser@example.com",
    role="user"
)

In [10]:
# Initialize Firebase Admin SDK
if not firebase_admin._apps:
    cred = credentials.Certificate("../.streamlit/tournalytics-firebase-adminsdk-dbcy8-e1435aaaa6.json")
    firebase_admin.initialize_app(cred)

# Initialize Firestore client
db = firestore.client()

In [11]:
import bcrypt
from firebase_admin import firestore
from datetime import datetime

# Initialize Firestore client
db = firestore.client()

def hash_password(plain_text_password):
    """
    Hash a plain-text password using bcrypt.
    """
    salt = bcrypt.gensalt()
    hashed_password = bcrypt.hashpw(plain_text_password.encode("utf-8"), salt)
    return hashed_password

def verify_password(plain_text_password, hashed_password):
    """
    Verify a plain-text password against a bcrypt hash.
    """
    return bcrypt.checkpw(plain_text_password.encode("utf-8"), hashed_password)

def create_user(username, password, role="user", league_id=None):
    """
    Create a new user in Firestore.
    """
    try:
        # Hash the password
        hashed_password = hash_password(password)
        user_doc = {
            "username": username,
            "hashed_password": hashed_password.decode("utf-8"),  # Store as string
            "role": role,
            "league_id": league_id,
            "created_at": datetime.now(),
        }
        db.collection("users").document(username).set(user_doc)
        return True
    except Exception as e:
        print(f"Error creating user: {e}")
        return False

def authenticate_user(username, password):
    """
    Authenticate a user against Firestore data.
    """
    try:
        user_ref = db.collection("users").document(username).get()
        if not user_ref.exists:
            print("User not found.")
            return None
        
        user_data = user_ref.to_dict()
        hashed_password = user_data.get("hashed_password")
        
        # Verify the password
        if verify_password(password, hashed_password.encode("utf-8")):
            return {"username": username, "role": user_data.get("role")}
        else:
            print("Invalid password.")
            return None
    except Exception as e:
        print(f"Error authenticating user: {e}")
        return None
